In [32]:
import numpy as np

In [163]:
from tqdm import tqdm

In [20]:
with open('data.txt') as f:
    raw_data = f.read().split('\n')
    data = [raw_data[0], raw_data[2:]]

In [241]:
test_case = ("initial state: #..#.#..##......###...###",
"""...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #""".strip().split('\n'))

In [242]:
test_ans = ".#....##....#####...#######....#.#..##."

In [243]:
def extract_pot_patterns(data):
    pot_patterns = set()
    for pattern in data[1]:
        if pattern[-1] == "#":
            pot_patterns.add(pattern[:5])
    return pot_patterns

In [338]:
def next_gen(pots, pot_patterns, num_gens=20, at_zero=False):
    if at_zero:
        initial_state = pots.replace("initial state: ", "")
        # have to add two empty pots to each end to properly match
        pad_right = "." * num_gens**2
        full_state = "..." + initial_state + pad_right
    else:
        full_state = pots
    
    next_state = ""
    for i in range(len(full_state)):
        # need to add empty pots "." to left of pot zero
        if i == 0:
            state = ".." + full_state[:i+3]
        elif i == 1:
            state = "." + full_state[i-1:i+3]
        else:
            state = full_state[i-2:i+3]
        if state in pot_patterns:
            next_state += "#"
        else:
            next_state += "."
    return next_state

In [339]:
def get_pots_after_ngens(pots, pot_patterns, num_gens=20, verbose=False, progress_bar=False):
    seen = set()
    gen = 0
    if progress_bar:
        if num_gens < 10000:
            pbar = tqdm(total=num_gens)
        else:
            one_percent = num_gens // 100
            pbar = tqdm(total=100)
    while gen < num_gens:
        if gen == 0:
            if verbose:
                print(f'{gen:2} :', "..." + pots.replace("initial state: ", ""))
            pots = next_gen(pots, pot_patterns, at_zero=True)
        else:
            pots = next_gen(pots, pot_patterns, at_zero=False)
        if pots in seen:
            print(f"Stopped early due to repeated cycle: {gen}")
            break
        seen.add(pots)
        gen += 1
        if verbose:
            print(f'{gen:2} :', pots)
        if progress_bar:
            if num_gens < 10000:
                pbar.update(1)
            else:
                if gen % one_percent == 0:
                    pbar.update(1)
    return pots

In [340]:
test_pots = get_pots_after_ngens(test_case[0], extract_pot_patterns(test_case), verbose=True, progress_bar=True)
assert test_pots[:len(test_ans)] == test_ans




  0%|          | 0/20 [00:00<?, ?it/s]


100%|██████████| 20/20 [00:00<00:00, 573.99it/s]

 0 : ...#..#.#..##......###...###
 1 : ...#...#....#.....#..#..#..#................................................................................................................................................................................................................................................................................................................................................................................................................
 2 : ...##..##...##....#..#..#..##...............................................................................................................................................................................................................................................................................................................................................................................................................
 3 : ..#.#...#..#.#....#..#..#...#................................................................

In [341]:
def score_pots(pots):
    has_pots = np.array([x == "#" for x in pots])
    points = np.array([i for i in range(-3, len(pots) - 3)])
    return np.sum(has_pots * points)

In [342]:
assert score_pots(test_pots) == 325

In [343]:
pots = get_pots_after_ngens(data[0], extract_pot_patterns(data))
assert score_pots(pots) == 2911

In [353]:
scores = {}
for trial in range(20, 250):
    pots = get_pots_after_ngens(data[0], extract_pot_patterns(data), num_gens=trial)
    scores[trial] = score_pots(pots)

In [355]:
import pandas as pd

In [357]:
scores2 = pd.Series(scores)

In [361]:
score_df = pd.DataFrame(scores2, columns=['score'])

In [362]:
score_df['diff_prior'] = score_df['score'] - score_df.score.shift()

In [363]:
score_df

,score,diff_prior
20,2911,NaN
21,2893,-18.0
22,2942,49.0
23,2963,21.0
24,2959,-4.0
25,2979,20.0
26,2906,-73.0
27,2995,89.0
28,2981,-14.0
29,3027,46.0


In [364]:
# settles on adding 50 every time
def score_part_two(num_gens=50000000000):
    pots = get_pots_after_ngens(data[0], extract_pot_patterns(data), num_gens=250)
    high = score_pots(pots)
    return (num_gens - 250) * 50 + high

In [366]:
score_part_two()

2500000000695

In [367]:
assert score_part_two() == 2500000000695